In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

# Data and evaluation
## Data loading

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/TAI_HW4_FAIRNESS/data.csv')

In [ ]:
df.head()

In [ ]:
df.describe()

In [ ]:
df.dtypes

In [ ]:
features = list(df.columns)
features.remove('income')

## Evaluation metrics

In [ ]:
MALE = 1
FEMALE = 0

def zemel_fairness(y_pred, genders):
  # zemel_fairness = P(Y=1 | gender=MALE) - P(Y=1 | gender=FEMALE)

  p_pos_gender0 = np.mean(y_pred[genders == MALE])
  p_pos_gender1 = np.mean(y_pred[genders == FEMALE])

  return p_pos_gender0 - p_pos_gender1

In [ ]:
def disparate_impact(y_pred, genders):
  # disparate_impact = P(Y=1 | gender=FEMALE) - P(Y=1 | gender=FEMALE)

  p_pos_gender0 = np.mean(y_pred[genders == MALE])
  p_pos_gender1 = np.mean(y_pred[genders == FEMALE])

  return p_pos_gender1 / p_pos_gender0

Testing the metrics on dataset:

In [ ]:
zemel_fairness(df['income'], df['gender'])

In [ ]:
disparate_impact(df['income'], df['gender'])

# Base model implementation

In [ ]:
train, test = train_test_split(df, test_size=0.3)

In [ ]:
x_train, y_train = train[features], train['income']
x_test, y_test = test[features], test['income']

In [ ]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
import xgboost as xgb

In [ ]:
clf = make_pipeline(StandardScaler(), xgb.XGBClassifier())
clf.fit(x_train, y_train)

In [ ]:
y_pred = clf.predict(x_test)

In [ ]:
from sklearn.metrics import classification_report

In [ ]:
print(classification_report(y_test, y_pred))

In [ ]:
zemel_fairness(y_pred, x_test['gender'])

In [ ]:
disparate_impact(y_pred, x_test['gender'])

# Unbiased model implementation

In [ ]:
prob_income = clf.predict_proba(df[features])
pred_income = clf.predict(df[features])

In [ ]:
dfm = df.copy()
dfm['prob_income'] = np.max(prob_income, axis=1)
dfm['pred_income'] = pred_income

In [ ]:
cp = dfm[(dfm['gender'] == MALE) & (dfm['income'] == 1)]
cp = cp.sort_values('prob_income')

In [ ]:
cd = dfm[(dfm['gender'] == FEMALE) & (dfm['income'] == 0)]
cd = cd.sort_values('prob_income', ascending=False)

In [ ]:
n_male = (dfm['gender'] == MALE).sum()
n_female = (dfm['gender'] == FEMALE).sum()
n_male_income = ((dfm['gender'] == MALE) & (dfm['pred_income'] == 1)).sum()
n_female_income = ((dfm['gender'] == FEMALE) & (dfm['pred_income'] == 1)).sum()

In [ ]:
N = ((n_female * n_male_income) - (n_male * n_female_income)) / (n_male + n_female)

In [ ]:
N

In [ ]:
for i in range(round(N)):
  cp_c = cp.iloc[i]['income']
  cd_c = cd.iloc[i]['income']

  cp.iloc[i]['income'] = cd_c
  cd.iloc[i]['income'] = cp_c

In [ ]:
new_df = pd.concat([cp, cd]).sample(frac=1)

In [ ]:
train, test = train_test_split(new_df, test_size=0.3)

In [ ]:
x_train, y_train = train[features], train['income']
x_test, y_test = test[features], test['income']

In [ ]:
clf = make_pipeline(StandardScaler(), xgb.XGBClassifier())
clf.fit(x_train, y_train)

In [ ]:
y_pred = clf.predict(x_test)

In [ ]:
from sklearn.metrics import classification_report

In [ ]:
print(classification_report(y_test, y_pred))

In [ ]:
zemel_fairness(y_pred, x_test['gender'])

In [ ]:
disparate_impact(y_pred, x_test['gender'])